In [1]:
import json
import numpy as np
import os
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup
import requests

# connect to poloniex's API
url = 'file:///home/abid/Desktop/public.html'

response = urllib.request.urlopen(url)
j=response.read().decode()
data=json.loads(j)
# for x in range(len(data)):
#      print(data[x])

    

# openUrl.close()
# d = json.loads(r.decode())
df = pd.DataFrame(data)
original_columns=[u'close', u'date', u'high', u'low', u'open']
new_columns = ['Close','Timestamp','High','Low','Open']
df = df.loc[:,original_columns]
df.columns = new_columns
df.to_csv('/home/abid/Desktop/bitcoin2015to2017.csv',index=None)


In [2]:
import numpy as np
import pandas as pd
 
class PastSampler:
    '''
    Forms training samples for predicting future values from past value
    '''
     
    def __init__(self, N, K, sliding_window = True):
        '''
        Predict K future sample using N previous samples
        '''
        self.K = K
        self.N = N
        self.sliding_window = sliding_window
 
    def transform(self, A):
        M = self.N + self.K     #Number of samples per row (sample + target)
        #indexes
        if self.sliding_window:
            I = np.arange(M) + np.arange(A.shape[0] - M + 1).reshape(-1, 1)
        else:
            if A.shape[0]%M == 0:
                I = np.arange(M)+np.arange(0,A.shape[0],M).reshape(-1,1)
                
            else:
                I = np.arange(M)+np.arange(0,A.shape[0] -M,M).reshape(-1,1)
            
        B = A[I].reshape(-1, M * A.shape[1], A.shape[2])
        ci = self.N * A.shape[1]    #Number of features per sample
        return B[:, :ci], B[:, ci:] #Sample matrix, Target matrix

#data file path
dfp = '/home/abid/Desktop/bitcoin2015to2017.csv'

#Columns of price data to use
columns = ['Close']
df = pd.read_csv(dfp)
time_stamps = df['Timestamp']
df = df.loc[:,columns]
original_df = pd.read_csv(dfp).loc[:,columns]


In [ ]:
file_name='/home/abid/Desktop/bitcoin2015to2017_close.h5'

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# normalization
for c in columns:
    df[c] = scaler.fit_transform(df[c].values.reshape(-1,1))
    
#Features are input sample dimensions(channels)
A = np.array(df)[:,None,:]
original_A = np.array(original_df)[:,None,:]
time_stamps = np.array(time_stamps)[:,None,None]

#Make samples of temporal sequences of pricing data (channel)
NPS, NFS = 256, 16         #Number of past and future samples
ps = PastSampler(NPS, NFS, sliding_window=False)
B, Y = ps.transform(A)
input_times, output_times = ps.transform(time_stamps)
original_B, original_Y = ps.transform(original_A)

import h5py
with h5py.File(file_name, 'w') as f:
    f.create_dataset("inputs", data = B)
    f.create_dataset('outputs', data = Y)
    f.create_dataset("input_times", data = input_times)
    f.create_dataset('output_times', data = output_times)
    f.create_dataset("original_datas", data=np.array(original_df))
    f.create_dataset('original_inputs',data=original_B)
    f.create_dataset('original_outputs',data=original_Y)


/home/abid/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
import pandas as pd
import numpy as numpy
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv1D, MaxPooling1D, LeakyReLU, PReLU
from keras.utils import np_utils
from keras.callbacks import CSVLogger, ModelCheckpoint
import h5py
import os
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session


# Make the program use only one GPU
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))


with h5py.File(''.join(['/home/abid/Desktop/bitcoin2015to2017_close.h5']), 'r') as hf:
    datas = hf['inputs'].value
    labels = hf['outputs'].value


output_file_name='/home/abid/Desktop/bitcoin2015to2017_close_CNN_2_relu'

step_size = datas.shape[1]
batch_size= 8
nb_features = datas.shape[2]

epochs = 100

#split training validation
training_size = int(0.8* datas.shape[0])
training_datas = datas[:training_size,:]
training_labels = labels[:training_size,:]
validation_datas = datas[training_size:,:]
validation_labels = labels[training_size:,:]
#build model

# 2 layers
model = Sequential()


model.add(Conv1D(activation='relu', input_shape=(step_size, nb_features), strides=3, filters=8, kernel_size=20))
model.add(Dropout(0.5))
model.add(Conv1D( strides=4, filters=nb_features, kernel_size=16))

'''
# 3 Layers
model.add(Conv1D(activation='relu', input_shape=(step_size, nb_features), strides=3, filters=8, kernel_size=8))
#model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Conv1D(activation='relu', strides=2, filters=8, kernel_size=8))
#model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Conv1D( strides=2, filters=nb_features, kernel_size=8))
# 4 layers
model.add(Conv1D(activation='relu', input_shape=(step_size, nb_features), strides=2, filters=8, kernel_size=2))
#model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Conv1D(activation='relu', strides=2, filters=8, kernel_size=2))
#model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Conv1D(activation='relu', strides=2, filters=8, kernel_size=2))
#model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Conv1D( strides=2, filters=nb_features, kernel_size=2))
'''

model.compile(loss='mse', optimizer='adam')
print(output_file_name)
model.fit(training_datas, training_labels,verbose=1, batch_size=batch_size,validation_data=(validation_datas,validation_labels), epochs = epochs, callbacks=[CSVLogger(output_file_name+'.csv', append=True),ModelCheckpoint(output_file_name+'-{epoch:02d}-{val_loss:.5f}.hdf5', monitor='val_loss', verbose=1,mode='min')])

Using TensorFlow backend.
/home/abid/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


/home/abid/Desktop/bitcoin2015to2017_close_CNN_2_relu
Train on 1020 samples, validate on 256 samples
Epoch 1/100
1020/1020 [==============================] - 0s 309us/step - loss: 5.4190e-04 - val_loss: 5.3847e-04

Epoch 00001: saving model to /home/abid/Desktop/bitcoin2015to2017_close_CNN_2_relu-01-0.00054.hdf5
Epoch 2/100
1020/1020 [==============================] - 0s 257us/step - loss: 2.5215e-04 - val_loss: 6.2437e-04

Epoch 00002: saving model to /home/abid/Desktop/bitcoin2015to2017_close_CNN_2_relu-02-0.00062.hdf5
Epoch 3/100
1020/1020 [==============================] - 0s 231us/step - loss: 2.1508e-04 - val_loss: 4.9727e-04

Epoch 00003: saving model to /home/abid/Desktop/bitcoin2015to2017_close_CNN_2_relu-03-0.00050.hdf5
Epoch 4/100
1020/1020 [==============================] - 0s 233us/step - loss: 1.9274e-04 - val_loss: 0.0022

Epoch 00004: saving model to /home/abid/Desktop/bitcoin2015to2017_close_CNN_2_relu-04-0.00217.hdf5
Epoch 5/100
1020/1020 [============================

1020/1020 [==============================] - 0s 254us/step - loss: 1.5561e-04 - val_loss: 0.0024

Epoch 00039: saving model to /home/abid/Desktop/bitcoin2015to2017_close_CNN_2_relu-39-0.00244.hdf5
Epoch 40/100
1020/1020 [==============================] - 0s 209us/step - loss: 1.3547e-04 - val_loss: 8.2304e-04

Epoch 00040: saving model to /home/abid/Desktop/bitcoin2015to2017_close_CNN_2_relu-40-0.00082.hdf5
Epoch 41/100
1020/1020 [==============================] - 0s 250us/step - loss: 1.3375e-04 - val_loss: 5.7776e-04

Epoch 00041: saving model to /home/abid/Desktop/bitcoin2015to2017_close_CNN_2_relu-41-0.00058.hdf5
Epoch 42/100
1020/1020 [==============================] - 0s 234us/step - loss: 1.3641e-04 - val_loss: 8.7344e-04

Epoch 00042: saving model to /home/abid/Desktop/bitcoin2015to2017_close_CNN_2_relu-42-0.00087.hdf5
Epoch 43/100
1020/1020 [==============================] - 0s 242us/step - loss: 1.3514e-04 - val_loss: 9.6583e-04

Epoch 00043: saving model to /home/abid/Deskto


Epoch 00077: saving model to /home/abid/Desktop/bitcoin2015to2017_close_CNN_2_relu-77-0.00381.hdf5
Epoch 78/100
1020/1020 [==============================] - 0s 248us/step - loss: 1.5574e-04 - val_loss: 6.1125e-04

Epoch 00078: saving model to /home/abid/Desktop/bitcoin2015to2017_close_CNN_2_relu-78-0.00061.hdf5
Epoch 79/100
1020/1020 [==============================] - 0s 246us/step - loss: 1.2751e-04 - val_loss: 9.7724e-04

Epoch 00079: saving model to /home/abid/Desktop/bitcoin2015to2017_close_CNN_2_relu-79-0.00098.hdf5
Epoch 80/100
1020/1020 [==============================] - 0s 219us/step - loss: 1.3108e-04 - val_loss: 4.8364e-04

Epoch 00080: saving model to /home/abid/Desktop/bitcoin2015to2017_close_CNN_2_relu-80-0.00048.hdf5
Epoch 81/100
1020/1020 [==============================] - 0s 294us/step - loss: 1.2768e-04 - val_loss: 4.8312e-04

Epoch 00081: saving model to /home/abid/Desktop/bitcoin2015to2017_close_CNN_2_relu-81-0.00048.hdf5
Epoch 82/100
1020/1020 [====================

In [ ]:
df = pd.read_csv('file:///home/abid/Desktop/test.csv')
df.head()

In [ ]:
from keras import applications
from keras.models import Sequential

from keras.models import Model
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.callbacks import CSVLogger
import tensorflow as tf
from scipy.ndimage import imread
import numpy as np
import random
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D, LeakyReLU
from keras import backend as K
import keras
from keras.callbacks import CSVLogger, ModelCheckpoint
from keras.backend.tensorflow_backend import set_session
from keras import optimizers
import h5py
from sklearn.preprocessing import MinMaxScaler
import os
import pandas as pd
# import matplotlib

import matplotlib.pyplot as plt

os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

with h5py.File(''.join(['/home/abid/Desktop/bitcoin2015to2017_close.h5']), 'r') as hf:
    datas = hf['inputs'].value
    labels = hf['outputs'].value
    input_times = hf['input_times'].value
    output_times = hf['output_times'].value
    original_inputs = hf['original_inputs'].value
    original_outputs = hf['original_outputs'].value
    original_datas = hf['original_datas'].value
    

scaler=MinMaxScaler()
#split training validation
training_size = int(0.8*datas.shape[0])
training_datas = datas[:training_size,:,:]
training_labels = labels[:training_size,:,:]
validation_datas = datas[training_size:,:,:]
validation_labels = labels[training_size:,:,:]
validation_original_outputs = original_outputs[training_size:,:,:]
validation_original_inputs = original_inputs[training_size:,:,:]
validation_input_times = input_times[training_size:,:,:]
validation_output_times = output_times[training_size:,:,:]

ground_true = np.append(validation_original_inputs,validation_original_outputs, axis=1)
ground_true_times = np.append(validation_input_times,validation_output_times, axis=1)
step_size = datas.shape[1]
batch_size= 8
nb_features = datas.shape[2]

model = Sequential()

# 2 layers
model.add(Conv1D(activation='relu', input_shape=(step_size, nb_features), strides=3, filters=8, kernel_size=20))
# model.add(LeakyReLU())
model.add(Dropout(0.25))
model.add(Conv1D( strides=4, filters=nb_features, kernel_size=16))
model.load_weights('/home/abid/Desktop/bitcoin2015to2017_close_CNN_2_relu-100-0.00063.hdf5')
model.compile(loss='mse', optimizer='adam')
print(validation_datas)


In [ ]:
predicted = model.predict(validation_datas)
print(predicted)

predicted = model.predict(0.36890888)

predicted_inverted = []

for i in range(original_datas.shape[1]):
    scaler.fit(original_datas[:,i].reshape(-1,1))
    predicted_inverted.append(scaler.inverse_transform(predicted[:,:,i]))
print(np.array(predicted_inverted))
#get only the close data
ground_true = ground_true[:,:,0].reshape(-1)
ground_true_times = ground_true_times.reshape(-1)
ground_true_times = pd.to_datetime(ground_true_times, unit='s')
# since we are appending in the first dimension
predicted_inverted = np.array(predicted_inverted)[0,:,:].reshape(-1)
print(np.array(predicted_inverted).shape)
validation_output_times = pd.to_datetime(validation_output_times.reshape(-1), unit='s')

In [ ]:
ground_true_df = pd.DataFrame()
ground_true_df['times'] = ground_true_times
ground_true_df['value'] = ground_true

prediction_df = pd.DataFrame()
prediction_df['times'] = validation_output_times
prediction_df['value'] = predicted_inverted

prediction_df = prediction_df.loc[(prediction_df["times"].dt.year == 2017 )&(prediction_df["times"].dt.month > 7 ),: ]
ground_true_df = ground_true_df.loc[(ground_true_df["times"].dt.year == 2017 )&(ground_true_df["times"].dt.month > 7 ),:]

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(ground_true_df.times,ground_true_df.value, label = 'Actual')
plt.plot(prediction_df.times,prediction_df.value,'ro', label='Predicted')
plt.legend(loc='upper left')
plt.show()

In [ ]:
def fit_lstm(reg):
    global training_datas, training_labels, batch_size, epochs,step_size,nb_features, units
    model = Sequential()
    model.add(CuDNNLSTM(units=units, bias_regularizer=reg, input_shape=(step_size,nb_features),return_sequences=False))
    model.add(Activation('tanh'))
    model.add(Dropout(0.2))
    model.add(Dense(output_size))
    model.add(LeakyReLU())
    model.compile(loss='mse', optimizer='adam')
    model.fit(training_datas, training_labels, batch_size=batch_size, epochs = epochs, verbose=0)
    return model